## Modules

In [31]:
import pandas as pd
import Levenshtein
import string, re

## Config

In [32]:
input_csv =  "results/test_llama_7b_finetuned_not_all_answerable_4bit.csv"
output_csv = input_csv.split('.')[0] + "_EXPANDED.csv"

## Evaluation

In [33]:
result_df = pd.read_csv(input_csv, sep=';', index_col=0)
result_df

,reference,question,correct_answer,answer_w_ref,answer_no_ref,rouge1_fmeasure
0,1943 novemberében Hitler Erwin Rommelt nevezte...,Miért Rommelt nevezte ki Hitler a francia védm...,Nincs elegendő adat a kérdés megválaszolásához.,Nincs elegendő adat a kérdés megválaszolásához...,A francia védművek felügyelője a támadás során...,0.956522
1,A gyermeket hét-nyolc éves korában az 1692–169...,Mikor veszítette el édesanyját Johann Sebastia...,A korabeli feljegyzések szerint a gyermek Bach...,"A fiú korán, mindössze kilencévesen veszítette...","1717-ben, 10 éves korában vesztette el édesany...",0.575758
2,A telepes lakásépítés a századfordulón egyre n...,Hány lakóháazt építettek a Wekerle-telepre?,A kor legismertebb példája a budapesti Wekerle...,Területét Wekerle Sándor pénzügyminiszter kezd...,A telepen 1973-ig 12550 lakóházt építettek fel...,0.422360
3,a szennyvíz és a szennyezett levegő tisztításá...,Kitől kell függetlennek lennie az OEKO-TEX STe...,Nincs elegendő adat a kérdés megválaszolásához.,az OEKO-TEX Szervezethez tartozó intézet függe...,Az OEKO-TEX STeP vizsgálatot végző auditornak ...,0.090909
4,1833. október 13-án Ilja Petrovics újból megnő...,Ilja Petrovics Csajkovszkijnak és Alekszandra...,"Házasságukból hét gyermek született, de első l...",1838. május 21-én itt született első fiúgyerme...,Ilja Petrovics Csajkovszkijnak és Alekszandra ...,0.229508
...,...,...,...,...,...,...
678,A Maine-öbölben a szervezett bálnafigyelésekko...,Milyen bálnák láthatók a Maine-öbölben?,Nincs elegendő adat a kérdés megválaszolásához.,A Maine-öbölben a szervezett bálnafigyelésekko...,A Maine-öbölben a következő bálnák élnek:\nCet...,0.137931
679,Az Intelmek Szent István király intelmei Imre ...,Mit jelent az admonitiones szó?,"A király fiához, a trónörökös Imre herceghez i...",Nincs elegendő adat a kérdés megválaszolásához...,"Az admonitio (latinul: admonitio, jelentése: t...",0.228571
680,Magyar Királyi Állami Főgimnázium (Tündérpalot...,Melyik útról nyílik a volt Magyar Királyi Álla...,Magyar Királyi Állami Főgimnázium (Tündérpalot...,Az oktatási minisztérium megbízásából Gaál Móz...,"A Károly utcából nyíló, 1903-ban emelt bejárat...",0.823529
681,"1981-ben adódott az első lehetőség arra, hogy ...",Mikor kapta meg Puskás a Kispest díszpolgára c...,1991-ben végleg Magyarországra költözött. Még ...,1992-ben Kispest díszpolgárává avatták.</s>,1958-ban kapta meg a Kispest díszpolgára címet...,0.500000


In [34]:
def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

def white_space_fix(text):
    return " ".join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()


def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))


def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [35]:
def find_most_similar_substring(string1, string2) -> (str, int):
    best_match = ""
    min_distance = float('inf')

    for i in range(len(string1) - len(string2) + 1):
        substring = string1[i:i+len(string2)]
        distance = Levenshtein.distance(substring, string2)

        if distance < min_distance:
            min_distance = distance
            best_match = substring

    return best_match, min_distance

In [36]:
failed_extraction_count = 0
all_extraction_count = 0
f1_col=[]

for index, row in result_df.iloc[:].iterrows():
    given_answer = row["answer_w_ref"].replace("</s>", '')
    f1_score = compute_f1(row["answer_w_ref"], row["correct_answer"])
    f1_col.append(f1_score)
    
    if given_answer not in row["reference"] and given_answer != "Nincs elegendő adat a kérdés megválaszolásához.":
        failed_extraction_count = failed_extraction_count + 1
        most_similar, distance = find_most_similar_substring(row["reference"], given_answer)
        print("Row: "+ str(index))
        print("[Question]\n" + '- ' + row["question"] + "\n")
        print("[Reference]\n" + row["reference"] + "\n")
        print("[Given answer]" + "[F1 score: " + str(f1_score) +"]\n" + given_answer + "\n")
        print("[Most similar part in reference]\n" + "[Levenshtein distance: "+ str(distance) +"]\n" + most_similar + '\n')
        print("[Correct answer]\n" + row["correct_answer"]) 
        print("-" * 100 + "\n\n")
    elif given_answer != "Nincs elegendő adat a kérdés megválaszolásához.":
        all_extraction_count = all_extraction_count + 1

result_df["f1_score"] =  f1_col
        

Row: 19
[Question]
- Hol alakult meg Közép-Európában az első egyetem?

[Reference]
Prága számos ismert kulturális látnivalónak ad otthont, amelyek közül sokan túlélték a 20. századi Európa erőszakát és pusztítását. A fő látnivalók közé tartozik a prágai vár, a Károly-híd, az Óváros tér a csillagászati ​​órával, a zsidó negyed, a Petřín-domb és Vyšehrad. 1992 óta Prága kiterjedt történelmi központja szerepel az UNESCO világörökségi listáján.
A városban több mint tíz múzeum található, valamint számos színház, galéria, mozi és egyéb kultúrális intézmények. Kiterjedt, modern tömegközlekedési rendszer köti össze a városrészeket egymással. Számos állami és magániskolának ad otthont, köztük a prágai Károly Egyetemnek, Közép-Európa legrégebbi egyetemének.
2017 óta a város évente több mint 8,5 millió nemzetközi látogatót fogadott, London, Párizs, Róma és Isztambul után az ötödik leglátogatottabb európai város volt.
Az „arany Prága” név valószínűleg I. Károly cseh király és (IV. Károly néven) né

In [37]:
print("Failed extraction percentage: "+ str((failed_extraction_count/(all_extraction_count+failed_extraction_count)) * 100) +' %')
print()

Failed extraction percentage: 14.175257731958762 %



In [38]:
result_df.head()

,reference,question,correct_answer,answer_w_ref,answer_no_ref,rouge1_fmeasure,f1_score
0,1943 novemberében Hitler Erwin Rommelt nevezte...,Miért Rommelt nevezte ki Hitler a francia védm...,Nincs elegendő adat a kérdés megválaszolásához.,Nincs elegendő adat a kérdés megválaszolásához...,A francia védművek felügyelője a támadás során...,0.956522,0.800000
1,A gyermeket hét-nyolc éves korában az 1692–169...,Mikor veszítette el édesanyját Johann Sebastia...,A korabeli feljegyzések szerint a gyermek Bach...,"A fiú korán, mindössze kilencévesen veszítette...","1717-ben, 10 éves korában vesztette el édesany...",0.575758,0.428571
2,A telepes lakásépítés a századfordulón egyre n...,Hány lakóháazt építettek a Wekerle-telepre?,A kor legismertebb példája a budapesti Wekerle...,Területét Wekerle Sándor pénzügyminiszter kezd...,A telepen 1973-ig 12550 lakóházt építettek fel...,0.422360,0.325000
3,a szennyvíz és a szennyezett levegő tisztításá...,Kitől kell függetlennek lennie az OEKO-TEX STe...,Nincs elegendő adat a kérdés megválaszolásához.,az OEKO-TEX Szervezethez tartozó intézet függe...,Az OEKO-TEX STeP vizsgálatot végző auditornak ...,0.090909,0.000000
4,1833. október 13-án Ilja Petrovics újból megnő...,Ilja Petrovics Csajkovszkijnak és Alekszandra...,"Házasságukból hét gyermek született, de első l...",1838. május 21-én itt született első fiúgyerme...,Ilja Petrovics Csajkovszkijnak és Alekszandra ...,0.229508,0.125000


In [39]:
result_df.to_csv(output_csv, index=False, sep=';')